In [230]:
import torch
import einops
from torch.nn import Module, Parameter, ParameterList
from torch import Tensor
from collections import namedtuple
from typing import List
import matplotlib.pyplot as plt

In [231]:
torch.manual_seed(0)

In [232]:
LayerInfo = namedtuple("LayerInfo", ["nodes", "memories"]) # num nodes, num mems per node

In [233]:
class SparseHopfield(Module):
  def __init__(self, receptive_fields, field_dim, layers: List[LayerInfo], t: int = 0, alpha: float=1.0, rho=1e-8):
    super().__init__()
    self.fields = receptive_fields
    self.field_dim = field_dim
    self.layers_info = layers
    self.t = t
    self.a = alpha
    self.rho = rho # used to prevent division by zero
    self.kwargs = {
        "receptive_fields": self.fields,
        "field_dim": self.field_dim,
        "layers": self.layers_info,
        "t": self.t,
        "alpha": self.a,
        "rho": self.rho
    }

    self.growth_threshold = self.a / (self.t + self.a)
    self.layers = []
    self.layer_counts = []

    self.create_layers()

  def reset_iteration(self, t: int = 0):
    self.t = t
    self.growth_threshold = self.a / (self.t + self.a)

  def update_iteration(self):
    self.t += 1
    self.growth_threshold = self.a / (self.t + self.a)

  def create_layers(self):
    for i in range(len(self.layers_info)):
      if i == 0: # assume we are in outer layer
        nodes, mems = self.layers_info[i]
        nodes = self.fields # we ignore what layers info says, we need to match the fields here
        self.layers_info[i] = LayerInfo(nodes=nodes, memories=mems)
        dim = self.field_dim
        counts = torch.ones(nodes, mems)
        mem_matrix = Parameter(torch.zeros((nodes, mems, dim)) + 0.5, requires_grad=False)
        self.layers.append(mem_matrix)
        self.layer_counts.append(counts)
      else:
        nodes, mems = self.layers_info[i]
        prev_nodes, prev_mems = self.layers_info[i - 1]
        assert prev_nodes % nodes == 0, f"At layer {i}, prev is {prev_nodes} but nodes is {nodes}"
        children_per_node = prev_nodes // nodes
        counts = torch.ones(nodes, mems)
        mem_matrix = Parameter(torch.zeros((nodes, children_per_node, mems, prev_mems)), requires_grad=False)
        self.layers.append(mem_matrix)
        self.layer_counts.append(counts)
      self.layers = ParameterList(self.layers)

  @staticmethod
  def maxi(x):
    max_indices = torch.argmax(x, dim=-1, keepdim=True)
    blank = torch.zeros_like(x)
    return torch.scatter(blank, -1, max_indices, torch.gather(x, -1, max_indices))

  @staticmethod
  def argmaxi(x, eps=1e-8):
    maxied = torch.abs(SparseHopfield.maxi(x))
    factors, indices = torch.max(maxied, dim=-1)
    factors = factors - eps
    return maxied / factors.unsqueeze(-1)

  @staticmethod
  def growth_argmaxi(x, counts, eps=1e-8, threshold=0.9):
    # for now we assume batch_size is 1
    batch_size, nodes, mems = x.shape
    normal_path = SparseHopfield.argmaxi(x, eps)
    _, indices = torch.min(counts, dim=-1, keepdim=True)
    indices_sg = indices.unsqueeze(0).expand(batch_size,-1,-1)
    growth_path = torch.zeros_like(normal_path)
    # values_of_interest = torch.gather(x, -1, indices_sg)
    # taking a closer look, it doesn't seem like the exact values from
    # the original should matter, we were just using it to divide
    # by itself (minus an eps) in the argmaxi to turn them into 1s,
    # to just setting it to 1 should be fine?
    values_of_interest = 1
    growth_path = torch.scatter(growth_path, -1, indices_sg, values_of_interest)
    growth_path = SparseHopfield.argmaxi(torch.abs(growth_path), eps)
    trigger_growth = torch.sum(x > threshold, dim=-1, keepdim=True) <= 0
    grown = torch.where(trigger_growth, growth_path, normal_path)

    _, activated_indices = torch.max(grown, dim=-1, keepdim=True)
    selected_counts = torch.gather(counts.unsqueeze(0).expand(batch_size, -1, -1), -1, activated_indices)
    zeroed_out_counts = selected_counts * ~trigger_growth
    updated_counts = torch.scatter(counts.unsqueeze(0).expand(batch_size, -1, -1), -1, activated_indices, zeroed_out_counts)
    updated_counts = updated_counts + grown
    updated_counts = einops.reduce(updated_counts, 'batch nodes mems -> nodes mems', 'sum')

    return grown, updated_counts.detach()

  @staticmethod
  def outer_forward_parallel(mems, xs, rho=1e-8):
    batch_size, fields, dim = xs.shape

    # if mems only has 3 dimensions, then we know it is missing the batch dim,
    # this is because the shape of mems should be
    # batch_size x fields x memories x dim
    m = mems - 0.5 if mems.dim() == 4 else (mems - 0.5).expand(batch_size, -1, -1, -1)
    x = xs - 0.5
    numerator = einops.einsum(m,x, 'batch fields memories dim, batch fields dim -> batch fields memories') * 0.5
    m_norm = torch.sqrt(einops.reduce(m ** 2, 'batch fields memories dim -> batch fields memories', 'sum'))
    x_norm = torch.sqrt(einops.reduce(x ** 2, 'batch field dim -> batch field', 'sum')).unsqueeze(-1)
    denom = m_norm * x_norm + rho
    return (numerator / denom) + 0.5

  @staticmethod
  def hidden_forward_parallel(hidden_mm, children_x, rho=1e-8):
    batch_size, total_children, children_mem_cols = children_x.shape
    if hidden_mm.dim() == 4:
      num_hidden_nodes, children_per_hidden, hidden_mems, children_mem_cols = hidden_mm.shape
      mm = hidden_mm.expand(batch_size, -1, -1, -1, -1)
    else:
      batch_size, num_hidden_nodes, children_per_hidden, hidden_mems, children_mem_cols = hidden_mm.shape
      mm = hidden_mm
    x = SparseHopfield.maxi(children_x).reshape(batch_size, num_hidden_nodes, children_per_hidden, children_mem_cols)
    propagation = einops.einsum(mm, x, 'batch hidden children h_mems c_mems, batch hidden children c_mems -> batch hidden h_mems')
    x_norm = torch.sqrt(einops.reduce(x**2,'batch hidden children c_mems -> batch hidden', 'sum')).unsqueeze(-1)
    norm_coeff = 1 / ((children_per_hidden * x_norm) + rho)
    return propagation * norm_coeff

  @staticmethod
  def down_prop_parallel(parent_h, parent_mm, child_h, coeff=0.5):
    # we assume `parent_h` has already been passed through argmaxi
    # in other words down_prop_parallel(argmaxi(parent_h), ...)
    batch_size, children, children_dim = child_h.shape
    if parent_mm.dim() == 4:
      mm = parent_mm.expand(batch_size, -1, -1, -1, -1)
    else:
      mm = parent_mm
    batch_size, parent_nodes, children_per_parent, parent_dim, child_dim = mm.shape
    argmaxi_parent_h = parent_h.unsqueeze(-2).expand(-1,-1, children_per_parent, -1)
    orig = child_h * coeff
    new = (1 - coeff) * einops.einsum(argmaxi_parent_h, mm, 'batch parents children pdim, batch parents children pdim cdim -> batch parents children cdim')
    new = new.reshape(batch_size, children, -1)
    return orig + new

  @staticmethod
  def pred(parent_down_prop, parent_mem_matrix):
    mm = parent_mem_matrix if parent_mem_matrix.dim() != 3 else parent_mem_matrix.unsqueeze(-3)
    nodes, children_per_node, memories, dim = mm.shape
    batch_size, nodes, memories = parent_down_prop.shape
    prediction = einops.einsum(mm, parent_down_prop, 'nodes children_per_node memories dim, batch nodes memories -> batch nodes children_per_node dim')
    prediction = prediction.reshape(batch_size, nodes * children_per_node, dim)
    return prediction

  @staticmethod
  def mem_delta(parent_down_prop, parent_mem_matrix, child_down_prop):
    """
    Calculates the delta that should be added to the given `parent_mem_matrix`
    to optimize the model. For example, to optimize the given matrix, use
    `new_mm = old_mm + lr * mem_delta(p_prop, old_mm, c_prop)`
    Note that the `lr` scaling should be done outside of this function

    Note also that, according to the paper / reference implementation, when
    calculating the matrix update for the outer layer, the child down prop is
    just the raw inputs, not one that is one-hot encoded
    """
    # shape is nodes memories dim, but we want
    # nodes children_per_node memories dim
    # in this case, we'll assume that children_per_node is 1
    mm = parent_mem_matrix if parent_mem_matrix.dim() != 3 else parent_mem_matrix.unsqueeze(-3)
    nodes, children_per_node, memories, dim = mm.shape
    batch_size, nodes, memories = parent_down_prop.shape

    prediction = SparseHopfield.pred(parent_down_prop, mm)
    error = child_down_prop - prediction
    error = error.reshape(batch_size, nodes, children_per_node, dim)
    delta = einops.einsum(error, parent_down_prop, 'batch nodes children_per_node dim, batch nodes memories -> nodes children_per_node memories dim')
    return delta.reshape(parent_mem_matrix.shape)

  def outer_up(self, sensor_input):
    outer_mm = self.layers[0]
    h_sub_l = SparseHopfield.outer_forward_parallel(outer_mm, sensor_input, self.rho)
    return h_sub_l

  def up(self, sensor_input):
    upwards = [self.outer_up(sensor_input)]
    for i in range(1, len(self.layers)):
      mm = self.layers[i]
      h_sub_l = SparseHopfield.hidden_forward_parallel(mm, upwards[i-1], self.rho)
      upwards.append(h_sub_l)
    return upwards

  def root_down(self, upwards, eps=1e-6):
    root_h_sub_l = upwards[-1]
    root_counts = self.layer_counts[-1]
    root_h_sub_l_star, root_counts = SparseHopfield.growth_argmaxi(root_h_sub_l, root_counts, eps, self.growth_threshold)
    self.layer_counts[-1] = root_counts
    return root_h_sub_l_star

  def down(self, upwards, eps=1e-6, coeff=0.5):
    downwards = [self.root_down(upwards, eps)]
    for i in range(len(upwards)-2, -1, -1):
      h_sub_l = upwards[i]
      counts = self.layer_counts[i]
      downed = SparseHopfield.down_prop_parallel(downwards[-1],self.layers[i+1],h_sub_l,coeff)
      h_sub_l_star, counts = SparseHopfield.growth_argmaxi(downed, counts, eps, self.growth_threshold)
      self.layer_counts[i] = counts
      downwards.append(h_sub_l_star)
    return downwards

  def pred_root_down(self, upwards, eps=1e-6):
    root_h_sub_l = upwards[-1]
    root_counts = self.layer_counts[-1]
    root_h_sub_l_star = SparseHopfield.argmaxi(root_h_sub_l, eps)
    return root_h_sub_l_star

  def pred_down(self, upwards, eps=1e-6, coeff=0.5):
    downwards = [self.pred_root_down(upwards, eps)]
    for i in range(len(upwards)-2, -1, -1):
      h_sub_l = upwards[i]
      counts = self.layer_counts[i]
      downed = SparseHopfield.down_prop_parallel(downwards[-1],self.layers[i+1],h_sub_l,coeff)
      h_sub_l_star = SparseHopfield.argmaxi(downed, eps)
      downwards.append(h_sub_l_star)
    return downwards

  def delta_outer(self, downwards, sensory_input):
    outer_mm = self.layers[0]
    outer_h_sub_l_star = downwards[-1]
    outer_delta = SparseHopfield.mem_delta(outer_h_sub_l_star, outer_mm, sensory_input)
    return outer_delta

  def delta(self, downwards, sensory_input):
    deltas = [self.delta_outer(downwards, sensory_input)]
    for i in range(1, len(downwards)):
      child_h_sub_l_star = downwards[len(downwards) - i]
      h_sub_l_star = downwards[len(downwards) - i - 1]
      mm = self.layers[i]
      delta = SparseHopfield.mem_delta(h_sub_l_star, mm, child_h_sub_l_star)
      deltas.append(delta)
    return deltas

  def optim_outer(self, deltas):
    outer_delta = deltas[0]
    outer_mm = self.layers[0]
    outer_counts = self.layer_counts[0]
    delta = outer_delta / (outer_counts).unsqueeze(-1)
    self.layers[0] = outer_mm + delta

  def optim(self, deltas):
    self.optim_outer(deltas)
    for i in range(1, len(deltas)):
      delta = deltas[i]
      mm = self.layers[i]
      count = self.layer_counts[i]
      delta = delta / (count).unsqueeze(-1).unsqueeze(-3)
      self.layers[i] = mm + delta
    self.update_iteration()

  def optimize(self, sensory_input, eps=1e-6, coeff=0.5):
    upwards = self.up(sensory_input)
    downwards = self.down(upwards, eps, coeff)
    deltas = self.delta(downwards, sensory_input)
    self.optim(deltas)

  def predict(self, sensory_input, eps=1e-6, coeff=0.5):
    with torch.no_grad():
      upwards = self.up(sensory_input)
      downwards = self.pred_down(upwards, eps, coeff)
      prediction = SparseHopfield.pred(downwards[-1],self.layers[0])
      return prediction

  def prediction_error(self, sensory_input, eps=1e-6, coeff=0.5):
    with torch.no_grad():
      pred = self.predict(sensory_input, eps, coeff)
      return torch.mean(torch.square(pred - sensory_input))

In [234]:
layers: List[LayerInfo] = [
    LayerInfo(nodes=-1, memories=15),
    LayerInfo(nodes=3, memories=15),
    LayerInfo(nodes=1, memories=15)
]
receptive_fields = 6
field_dim = 8

In [235]:
net = SparseHopfield(receptive_fields=receptive_fields,field_dim=field_dim,layers=layers, alpha=3.0)

In [236]:
def train(net, data):
    batch_size, fields, dim = data.shape
    for i in range(batch_size):
        net.optimize(data[i].unsqueeze(0))

def loss(net, data):
    batch_size, fields, dim = data.shape
    total_loss = 0
    errors = []
    for i in range(batch_size):
        err = net.prediction_error(data[i].unsqueeze(0))
        errors.append(err)
        total_loss += err
    return total_loss / batch_size, errors

In [237]:
def plot_tensors(tensor_dict):
    # given a dictionary of tensors, plot them, the key will be used as the titles
    num_tensors = len(tensor_dict)
    fig, axes = plt.subplots(1, num_tensors, figsize=(6 * num_tensors, 6))

    if num_tensors == 1:
        axes = [axes]

    for ax, (title, tensor) in zip(axes, tensor_dict.items()):
        ax.imshow(tensor.squeeze(0))
        ax.set_title(title)

    plt.show()

In [238]:
batch_size = 5

In [239]:
sensory_input = torch.randn(batch_size, receptive_fields, field_dim)

In [240]:
loss(net, sensory_input)

(tensor(1.1007),
 [tensor(1.1588),
  tensor(1.3570),
  tensor(1.2545),
  tensor(0.9086),
  tensor(0.8244)])

In [241]:
train(net, sensory_input)

In [242]:
loss(net, sensory_input)

(tensor(0.0038),
 [tensor(1.0979e-12),
  tensor(1.3024e-12),
  tensor(0.0095),
  tensor(9.1325e-13),
  tensor(0.0095)])

In [243]:
def corruption_replace(data, noise_level=0.1):
    # corrupts data based on noise level, each value has a `noise_level` chance of being replaced
    mask = torch.rand_like(data) < noise_level
    noise = torch.randn_like(data)
    return torch.where(mask, noise, data)

In [244]:
def corruption_additive(data, noise_level=0.1):
    # corrupts data based on noise level, each value has a `noise_level` chance of being added to it
    mask = torch.rand_like(data) < noise_level
    noise = torch.randn_like(data)
    return data + torch.where(mask, noise, torch.zeros_like(data))

In [245]:
def corruption_revolving_lantern(data, noise_level=0.1, right_to_left=False):
    # corrupts data based on noise level. Starting from either the right or left, it will
    # replace the first `noise_level` percentage of the data with noise and then return
    # the data. If `right_to_left` is True, it will start from the right, otherwise the left
    batch, rows, cols = data.shape
    new_data = data.clone()
    noise_cols = int(cols * noise_level)
    noise_cols = min(noise_cols, cols)
    noise = torch.randn(batch, rows, noise_cols)
    if right_to_left:
        new_data[:, :, -noise_cols:] = noise
    else:
        new_data[:, :, :noise_cols] = noise
    return new_data

In [246]:
def blackouts(data, noise_level=0.1):
    # corrupts data based on noise level. It will randomly blackout a percentage of the data
    # based on the noise level
    mask = torch.rand_like(data) < noise_level
    return torch.where(mask, torch.zeros_like(data), data)

In [247]:
def blackout_revolving_lantern(data, noise_level=0.1, right_to_left=False):
    # corrupts data based on noise level. Starting from either the right or left, it will
    # replace the first `noise_level` percentage of the data with zeros and then return
    # the data. If `right_to_left` is True, it will start from the right, otherwise the left
    batch, rows, cols = data.shape
    new_data = data.clone()
    noise_cols = int(cols * noise_level)
    noise_cols = min(noise_cols, cols)
    if right_to_left:
        new_data[:, :, -noise_cols:] = torch.zeros_like(new_data[:, :, -noise_cols:])
    else:
        new_data[:, :, :noise_cols] = torch.zeros_like(new_data[:, :, :noise_cols])
    return new_data